In [1]:
import pandas as pd

df = pd.read_json('data.json')
print(df.head)

<bound method NDFrame.head of                          _id runType  runNumber  \
0   64c6b47da62845d8d7b8c75e    lego          0   
1   64c6b61ea62845d8d7b8c766    lego          1   
2   64c6b7bea62845d8d7b8c76f    lego          2   
3   64c6b95fa62845d8d7b8c777    lego          3   
4   64c6bb00a62845d8d7b8c77f    lego          4   
5   64c6bca1a62845d8d7b8c785    lego          5   
6   64c6be42a62845d8d7b8c78c    lego          6   
7   64c6bfe3a62845d8d7b8c792    lego          7   
8   64c6c183a62845d8d7b8c79b    lego          8   
9   64c6c324a62845d8d7b8c7a1    lego          9   
10  64c6c4c5a62845d8d7b8c7a6    lego         10   
11  64c6c666a62845d8d7b8c7aa    lego         11   
12  64c6c807a62845d8d7b8c7ad    lego         12   
13  64c6c9a7a62845d8d7b8c7b0    lego         13   
14  64c6cb48a62845d8d7b8c7b3    lego         14   
15  64c6cce9a62845d8d7b8c7b6    lego         15   
16  64c6ce8aa62845d8d7b8c7b8    lego         16   
17  64c6d02aa62845d8d7b8c7ba    lego         17   
1

In [2]:
title = str(df['runType'][0]) + ' ' + str(df['sliders'][0])
fileName = str(df['runType'][0]) + ' ' + str(df['sliders'][0]) 
fileName = fileName.replace(' ', '_')
fileName = fileName.replace('{', '')
fileName = fileName.replace('}', '')
fileName = fileName.replace(':', '')
fileName = fileName.replace(',', '')
fileName = fileName.replace('\'', '')
print(fileName)

lego_cohesion_0.25_alignment_0.075_separation_25


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [4]:
def pointDistance(one, two):
    return math.sqrt((one['x'] - two['x']) ** 2 + (one['y'] - two['y']) ** 2)

In [5]:
def AverageDistance():
    aveDistance = {}

    for index, run in df.iterrows():
        preyList = run['preyList']
        positionList = run['positions']
        preyArr = []
        wolfArr = []
        closestDistance = []

        for i in range(len(preyList)):
            if (preyList[i] == True):
                preyArr.append(positionList[i])
            else :
                wolfArr.append(positionList[i])
        
        for tick in range(len(wolfArr[0])):
            thisTickTotal = 0
            for wolf in range(len(wolfArr)):
                closestSheep = pointDistance(wolfArr[wolf][tick], preyArr[0][tick])
                for sheep in range(1, len(preyArr)):
                    thisSheep = pointDistance(wolfArr[wolf][tick], preyArr[sheep][tick])
                    closestSheep = min(closestSheep, thisSheep)
                thisTickTotal += closestSheep
            closestDistance.append(thisTickTotal / len(wolfArr))
        aveDistance[index] = closestDistance
    return aveDistance

def SheepCaught():
    aveCaught = {}

    for index, run in df.iterrows():
        preyList = run['preyList']
        positionList = run['positions']
        preyArr = []
        wolfArr = []
        sheepC = []

        for i in range(len(preyList)):
            if (preyList[i] == True):
                preyArr.append(positionList[i])
            else :
                wolfArr.append(positionList[i])
        
        for tick in range(len(wolfArr[0])):
            thisTickTotal = 0
            for wolf in range(len(wolfArr)):
                for sheep in range(1, len(preyArr)):
                    thisSheep = pointDistance(wolfArr[wolf][tick], preyArr[sheep][tick])
                    thisTickTotal += 1 if thisSheep < 25 else 0
            sheepC.append(float(thisTickTotal / len(wolfArr)))
        aveCaught[index] = sheepC
    return aveCaught

def WolfDistance():
    wolfDistance = {}

    for index, run in df.iterrows():
        preyList = run['preyList']
        positionList = run['positions']
        wolfArr = []
        closestDistance = []

        for i in range(len(preyList)):
            if (preyList[i] == False):
                wolfArr.append(positionList[i])
        
        for tick in range(len(wolfArr[0])):
            thisTickTotal = 0
            for wolf in range(len(wolfArr)):
                closestSheep = pointDistance(wolfArr[wolf][tick], wolfArr[0][tick])
                for sheep in range(1, len(wolfArr)):
                    thisSheep = pointDistance(wolfArr[wolf][tick], wolfArr[sheep][tick])
                    if (thisSheep != 0):
                        closestSheep = min(closestSheep, thisSheep)
                thisTickTotal += closestSheep
            closestDistance.append(thisTickTotal / len(wolfArr))
        wolfDistance[index] = closestDistance
    return wolfDistance

def WeakDistance():
    weakDistance = {}

    for index, run in df.iterrows():
        preyList = run['preyList']
        positionList = run['positions']
        weak = run['positions'][0]
        wolfArr = []
        closestDistance = []

        for i in range(len(preyList)):
            if (preyList[i] == False):
                wolfArr.append(positionList[i])
        
        for tick in range(len(wolfArr[0])):
            closestWolf = pointDistance(wolfArr[0][tick], weak[tick])
            for wolf in range(len(wolfArr)):
                thisWolf = pointDistance(wolfArr[wolf][tick], weak[tick])
                closestWolf = min(closestWolf, thisWolf)
            closestDistance.append(closestWolf)
        weakDistance[index] = closestDistance
    return weakDistance

In [6]:
def graphAveDPlot(dict):
    # Plotting all average closest distance of all runs
    for i in dict:
        plt.scatter(range(len(dict[i])), dict[i], alpha=0.3, marker='.', s=1)

    # Initialize empty dictionaries to store the mean, min, and max distances for each timestamp
    mean_distances_per_timestamp = []
    min_distances_per_timestamp = []
    max_distances_per_timestamp = []

    # Iterate over the timestamps in the first run to calculate statistics for each timestamp
    for timestamp_idx, _ in enumerate(dict[0]):
        # Collect distances at the current timestamp from all runs
        distances_at_timestamp = [dict[run_num][timestamp_idx] for run_num in dict
                                if timestamp_idx < len(dict[run_num])]
        
        # Calculate mean, min, and max distances at the current timestamp
        mean_distance = np.mean(distances_at_timestamp)
        min_distance = np.min(distances_at_timestamp)
        max_distance = np.max(distances_at_timestamp)
        
        # Store the statistics for the current timestamp in the respective dictionaries
        mean_distances_per_timestamp.append(mean_distance)
        min_distances_per_timestamp.append(min_distance)
        max_distances_per_timestamp.append(max_distance)

    # Add a line for the overall mean
    plt.plot(mean_distances_per_timestamp, color='black', linestyle='-', label='Overall Mean')

    # Add a line for the overall minimum
    plt.plot(min_distances_per_timestamp, color='blue', linestyle='-', label='Overall Minimum')

    # Add a line for the overall maximum
    plt.plot(max_distances_per_timestamp, color='red', linestyle='-', label='Overall Maximum')

    plt.xlabel('Per 100 Ticks')
    plt.ylabel('Closest Wolf-Sheep Distance')
    plt.title('Average Closest Distance ' + title, fontsize = 10)
    # Add a legend
    plt.legend()

    plt.savefig('graphs/Ave_Min_Dist/' + fileName + '.png')
    plt.clf()

In [7]:
def graphAveSCPlot(dict):
    # Plotting all average closest distance of all runs
    for i in dict:
        plt.scatter(range(len(dict[i])), dict[i], alpha=0.3, marker='.', s=1)

    # Initialize empty dictionaries to store the mean, min, and max distances for each timestamp
    mean_distances_per_timestamp = []
    min_distances_per_timestamp = []
    max_distances_per_timestamp = []

    # Iterate over the timestamps in the first run to calculate statistics for each timestamp
    for timestamp_idx, _ in enumerate(dict[0]):
        # Collect distances at the current timestamp from all runs
        distances_at_timestamp = [dict[run_num][timestamp_idx] for run_num in dict
                                if timestamp_idx < len(dict[run_num])]
        
        # Calculate mean, min, and max distances at the current timestamp
        mean_distance = np.mean(distances_at_timestamp)
        min_distance = np.min(distances_at_timestamp)
        max_distance = np.max(distances_at_timestamp)
        
        # Store the statistics for the current timestamp in the respective dictionaries
        mean_distances_per_timestamp.append(mean_distance)
        min_distances_per_timestamp.append(min_distance)
        max_distances_per_timestamp.append(max_distance)
        
    # Add a line for the overall mean
    plt.plot(mean_distances_per_timestamp, color='black', linestyle='-', label='Overall Mean')

    # Add a line for the overall minimum
    plt.plot(min_distances_per_timestamp, color='blue', linestyle='-', label='Overall Minimum')

    # Add a line for the overall maximum
    plt.plot(max_distances_per_timestamp, color='red', linestyle='-', label='Overall Maximum')

    plt.xlabel('Per 100 Ticks')
    plt.ylabel('Number of Sheep Caught')
    plt.title('Average Sheep Caught ' + title, fontsize = 10)
    # Add a legend
    plt.legend()

    plt.savefig('graphs/Sheep_Caught/' + fileName + '.png')
    plt.clf()

In [8]:
def graphwolfDPlot(dict):
    # Plotting all average closest distance of all runs
    for i in dict:
        plt.scatter(range(len(dict[i])), dict[i], alpha=0.3, marker='.', s=1)

    # Initialize empty dictionaries to store the mean, min, and max distances for each timestamp
    mean_distances_per_timestamp = []
    min_distances_per_timestamp = []
    max_distances_per_timestamp = []

    # Iterate over the timestamps in the first run to calculate statistics for each timestamp
    for timestamp_idx, _ in enumerate(dict[0]):
        # Collect distances at the current timestamp from all runs
        distances_at_timestamp = [dict[run_num][timestamp_idx] for run_num in dict
                                if timestamp_idx < len(dict[run_num])]
        
        # Calculate mean, min, and max distances at the current timestamp
        mean_distance = np.mean(distances_at_timestamp)
        min_distance = np.min(distances_at_timestamp)
        max_distance = np.max(distances_at_timestamp)
        
        # Store the statistics for the current timestamp in the respective dictionaries
        mean_distances_per_timestamp.append(mean_distance)
        min_distances_per_timestamp.append(min_distance)
        max_distances_per_timestamp.append(max_distance)

    # Add a line for the overall mean
    plt.plot(mean_distances_per_timestamp, color='black', linestyle='-', label='Overall Mean')

    # Add a line for the overall minimum
    plt.plot(min_distances_per_timestamp, color='blue', linestyle='-', label='Overall Minimum')

    # Add a line for the overall maximum
    plt.plot(max_distances_per_timestamp, color='red', linestyle='-', label='Overall Maximum')

    plt.xlabel('Per 100 Ticks')
    plt.ylabel('Closest Wolf-Wolf Distance')
    plt.title('Average Closest Distance ' + title, fontsize = 10)
    # Add a legend
    plt.legend()

    plt.savefig('graphs/Ave_Min_Wolf_Dist/' + fileName + '.png')
    plt.clf()

In [9]:
def graphweakDPlot(dict):
    # Plotting all average closest distance of all runs
    for i in dict:
        plt.scatter(range(len(dict[i])), dict[i], alpha=0.3, marker='.', s=1)

    # Initialize empty dictionaries to store the mean, min, and max distances for each timestamp
    mean_distances_per_timestamp = []
    min_distances_per_timestamp = []
    max_distances_per_timestamp = []

    # Iterate over the timestamps in the first run to calculate statistics for each timestamp
    for timestamp_idx, _ in enumerate(dict[0]):
        # Collect distances at the current timestamp from all runs
        distances_at_timestamp = [dict[run_num][timestamp_idx] for run_num in dict
                                if timestamp_idx < len(dict[run_num])]
        
        # Calculate mean, min, and max distances at the current timestamp
        mean_distance = np.mean(distances_at_timestamp)
        min_distance = np.min(distances_at_timestamp)
        max_distance = np.max(distances_at_timestamp)
        
        # Store the statistics for the current timestamp in the respective dictionaries
        mean_distances_per_timestamp.append(mean_distance)
        min_distances_per_timestamp.append(min_distance)
        max_distances_per_timestamp.append(max_distance)

    # Add a line for the overall mean
    plt.plot(mean_distances_per_timestamp, color='black', linestyle='-', label='Overall Mean')

    # Add a line for the overall minimum
    plt.plot(min_distances_per_timestamp, color='blue', linestyle='-', label='Overall Minimum')

    # Add a line for the overall maximum
    plt.plot(max_distances_per_timestamp, color='red', linestyle='-', label='Overall Maximum')

    plt.xlabel('Per 100 Ticks')
    plt.ylabel('Closest Wolf-WSheep Distance')
    plt.title('Average Closest Distance ' + title, fontsize = 10)
    # Add a legend
    plt.legend()

    plt.savefig('graphs/Ave_Min_WSheep_Dist/' + fileName + '.png')
    plt.clf()

In [10]:
aveD = AverageDistance()
aveSC = SheepCaught()
wolfD = WolfDistance()
weakD = WeakDistance()
graphAveDPlot(aveD)
graphAveSCPlot(aveSC)
graphwolfDPlot(wolfD)
graphweakDPlot(weakD)

<Figure size 640x480 with 0 Axes>